### Agent-Lab: Multi-Agent -> Coder

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [1]:
%%capture
import json
import os
import nest_asyncio
from IPython.display import Markdown, display
from dotenv import load_dotenv
from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest

os.chdir("..")
load_dotenv()
nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### XAI Grok Coder

In [2]:
# Create Workflow
xai_agent = experiment_utils.create_xai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="grok-code-fast", api_key=os.getenv("XAI_API_KEY")
)
xai_coder_agent = container.coordinator_planner_supervisor_agent()
xai_workflow_builder = xai_coder_agent.get_workflow_builder(xai_agent["id"])
xai_workflow = xai_workflow_builder.compile(checkpointer=checkpointer)

In [3]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading. Make sure to include code examples in the final report.",
    agent_id=xai_agent["id"],
)

inputs = xai_coder_agent.get_input_params(message, schema="public")
config = xai_coder_agent.get_config(xai_agent["id"])
result = xai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = xai_coder_agent.format_response(result)

In [4]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user requests generating a simple Hello World example using FastAPI in Python, presented as a text report with markdown formatting, including code examples, without needing to set up libraries.",
  "title": "Generate FastAPI Hello World Code Example",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Generate FastAPI Hello World Code",
      "description": "Use coder to generate a simple Hello World application code using FastAPI in Python and output it as a Markdown report with code examples."
    },
    {
      "agent_name": "reporter",
      "title": "Compile Final Report",
      "description": "Use reporter to create a professional final report incorporating the generated code from the coder step, ensuring markdown formatting for readability and including code examples."
    }
  ]
}
```

In [5]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

# FastAPI Hello World Example

## Introduction
This report provides a simple "Hello World" example using FastAPI, a modern, fast (high-performance) web framework for building APIs with Python 3.7+ based on standard Python type hints. FastAPI is built on top of Starlette for the web parts and Pydantic for the data parts.

The example demonstrates a basic FastAPI application that responds to a GET request at the root endpoint ("/") with a JSON message.

**Assumptions:**
- Python 3.7+ is installed.
- FastAPI is installed (e.g., via `pip install fastapi`).
- Uvicorn is used as the ASGI server (install via `pip install uvicorn[standard]`).

## Code Example
Below is the complete Python code for the Hello World application. Save this as `main.py` (or any preferred filename).

```python
from fastapi import FastAPI

# Create a FastAPI instance
app = FastAPI()

# Define a route for the root endpoint
@app.get("/")
async def read_root():
    return {"Hello": "World"}
```

### Explanation of the Code:
- **Import FastAPI**: Imports the `FastAPI` class from the `fastapi` module.
- **App Instance**: Creates an instance of `FastAPI`, which serves as the main application object.
- **Route Decorator**: Uses `@app.get("/")` to define a GET endpoint at the root path. The function `read_root` is asynchronous and returns a dictionary, which FastAPI automatically converts to JSON.
- **Return Value**: The function returns a simple JSON response `{"Hello": "World"}`.

## How to Run the Application
1. Save the code above to a file, e.g., `main.py`.
2. Run the application using Uvicorn:
   ```
   uvicorn main:app --reload
   ```
   - `main` refers to the filename (without `.py`).
   - `app` is the FastAPI instance name.
   - `--reload` enables auto-reload on code changes (useful for development).
3. Open a web browser and navigate to `http://127.0.0.1:8000/`. You should see the JSON response: `{"Hello": "World"}`.
4. FastAPI automatically generates interactive API documentation at `http://127.0.0.1:8000/docs` (Swagger UI).

## Additional Notes
- FastAPI supports asynchronous operations, making it suitable for high-performance applications.
- For production, consider using a reverse proxy like Nginx and run Uvicorn with multiple workers.
- Error handling, validation, and more advanced features can be added as needed.

This example is minimal and ready to run. If you need extensions (e.g., adding more routes or middleware), let me know!

---
### Anthropic Coder

In [6]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-3-5-haiku-latest", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_coder_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_coder_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [7]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading. Make sure to include code examples in the final report.",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_coder_agent.get_input_params(message, schema="public")
config = anthropic_coder_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_coder_agent.format_response(result)

In [8]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "I'll use the coder agent to generate a simple FastAPI Hello World example, focusing on creating a clean, readable code snippet with a markdown-formatted report.",
  "title": "Generate FastAPI Hello World Python Code",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Create FastAPI Hello World Code",
      "description": "Generate a simple FastAPI application with a single endpoint that returns a hello world message. The code will demonstrate basic FastAPI routing and return functionality."
    },
    {
      "agent_name": "reporter",
      "title": "Document FastAPI Hello World Example",
      "description": "Prepare a professional report with the generated code, including explanation and markdown-formatted code blocks"
    }
  ]
}
```

In [9]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**



Here's a comprehensive report on creating a simple Hello World application with FastAPI:

# FastAPI Hello World Example

## Overview
This example demonstrates a minimal FastAPI application that returns a "Hello, World!" message when accessed.

## Code Example

```python
from fastapi import FastAPI

# Create a FastAPI application instance
app = FastAPI()

# Define a root endpoint that returns a hello world message
@app.get("/")
async def read_root():
    return {"message": "Hello, World!"}
```

## Key Components Explained

1. **Imports**
   - `from fastapi import FastAPI`: Imports the main FastAPI class to create the application

2. **Application Initialization**
   - `app = FastAPI()`: Creates an instance of the FastAPI application

3. **Route Definition**
   - `@app.get("/")`: Decorator that defines a GET endpoint at the root path
   - `async def read_root()`: Asynchronous function that handles the root endpoint
   - Returns a dictionary with a "Hello, World!" message

## Running the Application
Typically, you would run this application using Uvicorn:
```bash
uvicorn main:app --reload
```

## Additional Endpoints (Optional)
You can easily add more routes to expand functionality:

```python
@app.get("/hello/{name}")
async def hello_name(name: str):
    return {"message": f"Hello, {name}!"}
```

## Best Practices
- Use async functions for better performance
- Return JSON-serializable objects
- Use type hints for better code readability and IDE support

**Note**: This is a minimal example to demonstrate FastAPI's basic routing and response mechanism.

---
### OpenAI Coder

In [10]:
# Create Workflow
openai_agent = experiment_utils.create_openai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="gpt-5-nano", api_key=os.getenv("OPENAI_API_KEY")
)
openai_coder_agent = container.coordinator_planner_supervisor_agent()
openai_workflow_builder = openai_coder_agent.get_workflow_builder(openai_agent["id"])
openai_workflow = openai_workflow_builder.compile(checkpointer=checkpointer)

In [11]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading.",
    agent_id=openai_agent["id"],
)

inputs = openai_coder_agent.get_input_params(message, schema="public")
config = openai_coder_agent.get_config(openai_agent["id"])
result = openai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = openai_coder_agent.format_response(result)

In [12]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "User request: generate a simple Hello World example using FastAPI in Python, provided as a Markdown-formatted code report. Plan: allocate the coding task to the coder to produce a minimal, self-contained script that can be saved as a single file (e.g., main.py) and run with uvicorn. The code will be shown in a Markdown code block for easy reading, with no installation steps included.",
  "title": "Plan to generate a markdown-formatted Python FastAPI Hello World script",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Create a minimal FastAPI hello-world Python script",
      "description": "Produce a self-contained Python script (suitable to save as main.py) that defines a FastAPI app with a single root endpoint returning a simple Hello World payload. Include a standard __main__ block to run the app via uvicorn (host 0.0.0.0, port 8000). Present the entire code as a Markdown-formatted code block (```python ... ```), with no extra setup instructions."
    }
  ]
}
```

In [13]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

# FastAPI Hello World (main.py)

A minimal FastAPI app that exposes a root endpoint returning a "Hello, World!" message.

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Hello, World!"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
```